# Model

In [1]:
from transformers import AutoModel, AutoTokenizer
import torch

model_name = "5CD-AI/Vintern-1B-v3_5"

model = AutoModel.from_pretrained(
  model_name,
  torch_dtype=torch.bfloat16,
  low_cpu_mem_usage=True,
  trust_remote_code=True,
  use_flash_attn=False,
).cuda()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False, padding='right')

config.json:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

configuration_internvl_chat.py:   0%|          | 0.00/3.88k [00:00<?, ?B/s]

configuration_intern_vit.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/5CD-AI/Vintern-1B-v3_5:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/5CD-AI/Vintern-1B-v3_5:
- configuration_internvl_chat.py
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_internvl_chat.py:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/9.02k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.38M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

# Dataset

In [29]:
from datasets import load_dataset
dataset = load_dataset('5CD-AI/Vietnamese-Multi-turn-Chat-Alpaca')

In [30]:
def prepare_dataset(sample):
    processed_sample = []
    for turn in sample['conversations']:
        processed_turn = {}
        if turn['from'] == 'human':
            processed_turn['role'] = 'user'
        elif turn['from'] == 'gpt':
            processed_turn['role'] = 'assistant'
        processed_turn['content'] = turn['value']
        processed_sample.append(processed_turn)
    return {'conversations':processed_sample}
    

In [31]:
# Apply dataset preparation
print("Preparing dataset...")
dataset = dataset['train'].map(prepare_dataset, remove_columns=['conversations', 'id'], num_proc=4)
print(f"Processed dataset sample: {dataset[0]['conversations']}")

# Apply chat template (no pixel_values here)
print("Applying chat template...")
dataset = dataset.map(lambda x: {'text': tokenizer.apply_chat_template(x['conversations'], tokenize=False)}, num_proc=4)
print(f"Formatted sample: {dataset[0]['text'][:200]}...")

# Tokenize dataset
def tokenize_function(example):
    tokenized = tokenizer(example['text'], padding='longest', truncation=True, max_length=512)
    # Store image_flags as a scalar (not tensor yet, to avoid pin_memory issue)
    tokenized['image_flags'] = 0  # Scalar integer for text-only
    return tokenized

print("Tokenizing dataset...")
tokenized_dataset = dataset.map(tokenize_function, batched=False, remove_columns=['conversations', 'text'], load_from_cache_file=True)

Preparing dataset...
Processed dataset sample: [{'content': 'Hãy chỉnh sửa câu này để ngắn gọn hơn mà không mất đi ý nghĩa: "Trận đấu là một thất bại nặng nề mặc dù thực tế là cả đội đã tập luyện trong nhiều tuần."', 'role': 'user'}, {'content': 'Nhiều tuần huấn luyện của đội đã dẫn đến một thất bại nặng nề.', 'role': 'assistant'}, {'content': 'Bạn có thể đề xuất một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của họ trong trận đấu tiếp theo không?', 'role': 'user'}, {'content': 'Chắc chắn, đây là một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của mình trong trận đấu tiếp theo: 1. Phân tích trận đấu trước bằng cách xem lại cảnh quay trận đấu để xác định điểm yếu và các lĩnh vực cần cải thiện. 2. Tăng cường độ và sự tập trung của các buổi tập để đảm bảo mọi cầu thủ đều được chuẩn bị đầy đủ về thể chất và tinh thần. 3. Luyện tập những kỹ năng cụ thể cần cải thiện, chẳng hạn như sút bóng hoặc chuyền bóng chính xác. 4. Phát triển các chiến lược và chiến thu

Map:   0%|          | 0/12697 [00:00<?, ? examples/s]

In [34]:
from transformers import DataCollatorForLanguageModeling

# Custom data collator to add dummy pixel_values
class CustomDataCollator(DataCollatorForLanguageModeling):
    def __call__(self, examples):
        batch = super().__call__(examples)
        batch_size = len(examples)
        # Add dummy pixel_values for the batch (shared tensor to save memory)
        batch['pixel_values'] = torch.zeros((batch_size, 3, 3, 448, 448), dtype=torch.float16).to("cuda")
        batch['image_flags'] = torch.tensor([ex['image_flags'] for ex in examples], dtype=torch.long).to("cuda")
        return batch

In [35]:
from transformers import TrainingArguments, Trainer
import os

# Disable W&B to avoid hangs
os.environ["WANDB_DISABLED"] = "true"

# Define training arguments
training_args = TrainingArguments(
    output_dir="./qwen2_finetuned",
    per_device_train_batch_size=1,  # Minimize memory usage
    gradient_accumulation_steps=8,  # Effective batch size = 8
    learning_rate=2e-5,
    num_train_epochs=1,  # Start with 1 epoch
    logging_steps=1,  # Log every step
    logging_dir="./logs",
    fp16=True,  # Mixed precision
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none",  # Disable all logging integrations
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=CustomDataCollator(tokenizer=tokenizer, mlm=False),
)

# Start fine-tuning
trainer.train()

RuntimeError: cannot pin 'torch.cuda.LongTensor' only dense CPU tensors can be pinned

In [21]:
help(model.forward)

Help on method forward in module transformers_modules.5CD-AI.Vintern-1B-v3_5.115975aca0407d3c54bb0beb8a1da54d31de3f20.modeling_internvl_chat:

forward(pixel_values: torch.FloatTensor, input_ids: torch.LongTensor = None, attention_mask: Optional[torch.Tensor] = None, position_ids: Optional[torch.LongTensor] = None, image_flags: Optional[torch.LongTensor] = None, past_key_values: Optional[List[torch.FloatTensor]] = None, labels: Optional[torch.LongTensor] = None, use_cache: Optional[bool] = None, output_attentions: Optional[bool] = None, output_hidden_states: Optional[bool] = None, return_dict: Optional[bool] = None) -> Union[Tuple, transformers.modeling_outputs.CausalLMOutputWithPast] method of transformers_modules.5CD-AI.Vintern-1B-v3_5.115975aca0407d3c54bb0beb8a1da54d31de3f20.modeling_internvl_chat.InternVLChatModel instance
    Define the computation performed at every call.
    
    Should be overridden by all subclasses.
    
    .. note::
        Although the recipe for forward pa